In [1]:
#!source myenv/bin/activate
#!python3 --version
!which python3
from tabulate import tabulate
import pandas as pd
from sklearn.model_selection import train_test_split
!python3 --version

/home/adnan/Desktop/training/.venv/bin/python3
Python 3.8.10


In [3]:
!python3 --version

Python 3.8.10


In [5]:
!ls

__pycache__                     test.log
checkpoint-best-acc2            test_glitch.jsonl
checkpoint-best-accc            test_glitch.jsonl.jsonl
codebert-classification         test_glitch_with.jsonl
enhance                         train.jsonl
epoch3                          train.sh
epoch6                          train_glitch.jsonl
inference.sh                    train_glitch_with.jsonl
lastansible                     untitled.txt
logs                            untitled1.txt
model.py                        valid.jsonl
newdata                         valid_glitch.jsonl
predictions.txt                 valid_glitch_with.jsonl
pupet_advance                   with2
puppet-Akond-walliams           withcontext
puppet_william_akond_programing withcontextClean
run-t.py                        withcontextPredicrion
run.py                          withoutcontextPredection
test.jsonl


In [2]:
# getting dataset collected from ansible deffect dataset
file_path = "./scripts_with_labels_preprocessed.csv"
dfn = pd.read_csv(file_path)

#testing 
print(dfn)


      File.ID                                     Script.Content  \
0           1  class logstash( $heap_memory_mb, $filter_worke...   
1           2  class cluster::haproxy::rsyslog ( $log_file = ...   
2           3  class keystone::logging( $use_syslog = $::os_s...   
3           4  class ssh::config { include users::root includ...   
4           5  define midonet::resources::network_creation( $...   
...       ...                                                ...   
1953     1954  define lma_collector::afd_filter ( $type, $clu...   
1954     1955      class { ::openstack_tasks::swift::keystone :}   
1955     1956  class deploystudio { include deploystudio::fil...   
1956     1957  define supervisord::supervise($command, $user,...   
1957     1958  class rsyslog { include packages::rsyslog case...   

      Defect.Label  Dataset.Name  
0                0             3  
1                1             2  
2                0             2  
3                1             1  
4       

In [42]:

ls

__pycache__/                     test.jsonl
checkpoint-best-acc2/            test.log
checkpoint-best-accc/            test_glitch.jsonl
codebert-classification/         test_glitch.jsonl.jsonl
enhance/                         test_glitch_with.jsonl
epoch3/                          train.jsonl
epoch6/                          train.sh
inference.sh                     train_glitch.jsonl
lastansible/                     train_glitch_with.jsonl
logs/                            untitled.txt
model.py                         untitled1.txt
newdata/                         valid.jsonl
predictions.txt                  valid_glitch.jsonl
pupet_advance/                   valid_glitch_with.jsonl
puppet-Akond-walliams/           with2/
puppet_new_akond_william/        withcontext/
puppet_william_akond_programing/ withcontextClean/
run-t.py                         withcontextPredicrion/
run.py                           withoutcontextPredection/


In [4]:
#removing duplicates and null
dfn.dropna(subset=["Script.Content"], inplace=True)

In [7]:

# Splitting into training and test, assuming train_id and test_id are defined
train, test = train_test_split(dfn, test_size=0.2, random_state=42)  # random_state ensures reproducibility

print(f"Total number of entries: {len(dfn)}")
print(f"Number of training entries: {len(train)}")
print(f"Number of test entries: {len(test)}")

NameError: name 'train_test_split' is not defined

In [5]:
import json
from tqdm import tqdm

def create_data(set_data='train', directory='./'):
    data_list = []  # Use a list to store data dictionaries

    if set_data == 'train':
        data_df = train
        print(data_df)
    elif set_data == 'valid':
        # Use validation set size based on train/test split
        validation_size = int(len(train) * 0.2)  # Assumes 20% validation
        data_df = train.iloc[validation_size:]  # Select validation set from remaining entries
    elif set_data == 'test':
        data_df = test

    print(f"Number of entries in {set_data} set: {len(data_df)}")

    # Iterate over the DataFrame and append data for "Before" and "After" snippets
    for index, row in tqdm(data_df.iterrows()):
        code_before = str(row["Script.Content"])
        label = row["Defect.Label"]

     

        # Check if code_before is not empty
        if code_before.strip():
            # Append a dictionary for "Before" code snippet
            data_list.append({
                'code': code_before,
                'label':label,
       
            })



    # Write data to a single JSONL file
    with open(f'{directory}/{set_data}.jsonl', 'w') as f:
        for item in data_list:
            json.dump(item, f)
            f.write('\n')  # Add newline after each JSON object

    return data_list

# Create JSONL files for training, validation, and test sets
train_data = create_data(set_data='train')
valid_data = create_data(set_data='valid')
test_data = create_data(set_data='test')

data = []

with open('./train.jsonl', 'r') as file:
    for line in file:
        # Parse JSON object from each line
        json_obj = json.loads(line)
        data.append(json_obj)

# Create DataFrame from the list of JSON objects
dfn = pd.DataFrame(data)
dfn.head(5)


      File.ID                                     Script.Content  \
907       908  class nova::conductor( $enabled = true, $manag...   
857       858  class keystone::federation::mellon ( $methods,...   
1116     1117  class contrail::webui { package { ensure => pr...   
670       671  class ceph::conf($args = {}, $defaults = {}) {...   
1206     1207                                   include ::ec2api   
...       ...                                                ...   
1130     1131  class vnc::ultravnc_ini { include packages::ul...   
1294     1295  class activemq( $version = .., $hostname = 127...   
860       861  class cdh::hadoop::namenode { class[cdh::hadoo...   
1459     1460  class cdh::hadoop( $namenode_hosts, $dfs_name_...   
1126     1127  class tripleo::profile::base::neutron::server ...   

      Defect.Label  Dataset.Name  
907              1             2  
857              1             2  
1116             0             2  
670              0             2  
1206    

1566it [00:00, 10355.33it/s]


Number of entries in valid set: 1253


1253it [00:00, 13095.77it/s]


Number of entries in test set: 392


392it [00:00, 13177.27it/s]


,code,label
0,"class nova::conductor( $enabled = true, $manag...",1
1,"class keystone::federation::mellon ( $methods,...",1
2,class contrail::webui { package { ensure => pr...,0
3,"class ceph::conf($args = {}, $defaults = {}) {...",0
4,include ::ec2api,0


In [8]:
import pandas as pd

# Option 1: Finding Longest String Length (if "Before" and "After" contain text)
if pd.api.types.is_string_dtype(dfn['code']):
    t= dfn['code'].apply(lambda x: len(x.split()))
    max_token_length = t.max()
    print(f"Maximum token length (longest string length) in the 'code' column: {max_token_length}")

Maximum token length (longest string length) in the 'code' column: 1866


In [ ]:
!git clone https://github.com/Ducanger/codebert-classification

Cloning into 'codebert-classification'...
Username for 'https://github.com': 

In [1]:
%cd ./codebert-classification/code/


/home/adnan/Desktop/training/codebert/codebert-classification/code


In [4]:
ls

__pycache__/                      test.jsonl*
checkpoint-best-acc2/             test.log*
checkpoint-best-accc/             test_glitch.jsonl*
codebert-classification/          test_glitch.jsonl.jsonl*
enhance/                          test_glitch_with.jsonl*
epoch3/                           train.jsonl*
epoch6/                           train.sh*
inference.sh*                     train_glitch.jsonl*
lastansible/                      train_glitch_with.jsonl*
logs/                             untitled.txt*
model.py*                         untitled1.txt*
newdata/                          valid.jsonl*
predictions.txt*                  valid_glitch.jsonl*
pupet_advance/                    valid_glitch_with.jsonl*
puppet-Akond-walliams/            with2/
puppet_new_akond_william/         withcontext/
puppet_william_akond_programing/  withcontextClean/
run-t.py*                         withcontextPredicrion/
run.py*                           withoutcontextPredection/


In [4]:
!pip3 install --pre torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/nightly/cpu


!python3 --version

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/nightly/cpu
Python 3.8.10


In [2]:
!pip3 install tqdm



In [1]:
!pip3 install transformers


In [6]:
!python3 run.py \
    --output_dir=./puppet_william_akond_programing \
    --tokenizer_name=microsoft/codebert-base \
    --model_name_or_path=microsoft/codebert-base \
    --do_train \
    --train_data_file='/home/adnan/Desktop/training/codebert/train.jsonl'\
    --eval_data_file='/home/adnan/Desktop/training/codebert/valid.jsonl' \
    --test_data_file='/home/adnan/Desktop/training/codebert/test.jsonl' \
    --num_train_epochs 60 \
    --block_size 512\
    --train_batch_size 8\
    --eval_batch_size  8\
    --learning_rate 5e-5 \
    --weight_decay 0.001    \
    --max_grad_norm 1.0 \
    --seed 123456       \
    --warmup_steps=2000



07/09/2024 11:08:38 - WARNING - __main__ -   device: cuda, n_gpu: 1
Vocabulary size: 50265
Training/evaluation parameters %s Namespace(adam_epsilon=1e-08, block_size=512, device=device(type='cuda'), do_eval=False, do_test=False, do_train=True, eval_batch_size=8, eval_data_file='/home/adnan/Desktop/training/codebert/valid.jsonl', learning_rate=5e-05, max_grad_norm=1.0, model_name_or_path='microsoft/codebert-base', n_gpu=1, num_train_epochs=60, output_dir='./puppet_william_akond_programing', seed=123456, test_data_file='/home/adnan/Desktop/training/codebert/test.jsonl', tokenizer_name='microsoft/codebert-base', train_batch_size=8, train_data_file='/home/adnan/Desktop/training/codebert/train.jsonl', warmup_steps=2000, weight_decay=0.001)
Token indices sequence length is longer than the specified maximum sequence length for this model (605 > 512). Running this sequence through the model will result in indexing errors
*** Example ***
label: 1
input_tokens: ['<s>', 'class', '_no', 'va', '::'

In [4]:
ls

__pycache__/                      test.jsonl*
checkpoint-best-acc2/             test.log*
checkpoint-best-accc/             test_glitch.jsonl*
codebert-classification/          test_glitch.jsonl.jsonl*
enhance/                          test_glitch_with.jsonl*
epoch3/                           train.jsonl*
epoch6/                           train.sh*
inference.sh*                     train_glitch.jsonl*
lastansible/                      train_glitch_with.jsonl*
logs/                             untitled.txt*
model.py*                         untitled1.txt*
newdata/                          valid.jsonl*
predictions.txt*                  valid_glitch.jsonl*
pupet_advance/                    valid_glitch_with.jsonl*
puppet-Akond-walliams/            with2/
puppet_new_akond_william/         withcontext/
puppet_william_akond_programing/  withcontextClean/
run-t.py*                         withcontextPredicrion/
run.py*                           withoutcontextPredection/


In [2]:
#testing 
!python run.py\
    --output_dir=./puppet_william_akond_programing\
    --tokenizer_name=microsoft/codebert-base\
    --model_name_or_path=microsoft/codebert-base\
    #--do_eval \
    --do_test \
    --train_data_file='/Users/adnan.alrawass/jupyter-1.0.0/project_snt/puppet_william_akond_programing/train.jsonl'\
    --eval_data_file='/Users/adnan.alrawass/jupyter-1.0.0/project_snt/puppet_william_akond_programing/valid.jsonl' \
    --test_data_file='/Users/adnan.alrawass/jupyter-1.0.0/project_snt/puppet_william_akond_programing/test.jsonl' \
    --num_train_epochs 300 \
    --block_size 512\
    --train_batch_size 128 \
    --eval_batch_size  128\
    --learning_rate 2e-5 \
    --weight_decay 0.0001    \
    --max_grad_norm 1.0 \
    --seed 123456       \
    --warmup_steps=100



python: can't open file 'run.py': [Errno 2] No such file or directory


In [22]:
from sklearn.metrics import precision_score, recall_score, accuracy_score
def calculate_metrics(predictions_file, labels_file):
  """Calculates precision, recall, and accuracy from prediction and label files.

  Args:
      predictions_file: Path to the file containing predictions (one per line).
      labels_file: Path to the file containing labels (one per line).

  Returns:
      A dictionary containing precision, recall, and accuracy values.
  """
  with open(predictions_file, 'r') as f:
    predictions = [int(line.strip()) for line in f]
  with open(labels_file, 'r') as f:
    labels = [int(line.strip()) for line in f]

  precision = precision_score(labels, predictions)
  recall = recall_score(labels, predictions)
  accuracy = accuracy_score(labels, predictions)

  return {
      "precision": precision,
      "recall": recall,
      "accuracy": accuracy
  }


In [13]:
#with cleanning with testing 
predictions_file = "./puppet_william_akond_programing/predictions.txt"
labels_file = "./puppet_william_akond_programing/labels.txt"
metrics = calculate_metrics(predictions_file, labels_file)
print("Metrics:")
print(f"Precision: {metrics['precision']:.4f}")
print(f"Recall: {metrics['recall']:.4f}")
print(f"Accuracy: {metrics['accuracy']:.4f}")

Metrics:
Precision: 0.7286
Recall: 0.6145
Accuracy: 0.6519


In [9]:
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score

def calculate_metrics(predictions_file, labels_file):
    # Load predictions and labels from files
    with open(predictions_file, 'r') as f:
        predictions = [int(line.strip()) for line in f]
    with open(labels_file, 'r') as f:
        labels = [int(line.strip()) for line in f]

    # Calculate AUC
    auc = roc_auc_score(labels, predictions)

    # Convert predictions to binary labels
    predictions_binary = [1 if pred >= 0.5 else 0 for pred in predictions]

    # Calculate other metrics
    precision = precision_score(labels, predictions_binary)
    recall = recall_score(labels, predictions_binary)
    f1 = f1_score(labels, predictions_binary)
    accuracy = accuracy_score(labels, predictions_binary)

    return {
        'auc': auc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'accuracy': accuracy
    }

# File paths
predictions_file = "./puppet_william_akond_programing/predictions.txt"
labels_file = "./puppet_william_akond_programing/labels.txt"

# Calculate metrics
metrics = calculate_metrics(predictions_file, labels_file)

# Print metrics
print("Metrics:")
print(f"AUC: {metrics['auc']:.4f}")
print(f"Precision: {metrics['precision']:.4f}")
print(f"Recall: {metrics['recall']:.4f}")
print(f"F1-score: {metrics['f1']:.4f}")
print(f"Accuracy: {metrics['accuracy']:.4f}")


Metrics:
AUC: 0.6576
Precision: 0.7286
Recall: 0.6145
F1-score: 0.6667
Accuracy: 0.6519


In [14]:
# Assuming you have your tokenizer loaded
# For example: tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Get the vocabulary of the tokenizer
vocab = tokenizer.get_vocab()

# Initialize variables to store token lengths, max length, and min length
token_lengths = []
max_token_length = 0
min_token_length = float('inf')  # Initialize min_token_length to positive infinity

# Iterate through the vocabulary
for token in vocab.keys():
    # Get the length of the current token
    token_length = len(token)
    
    # Update the max token length if necessary
    max_token_length = max(max_token_length, token_length)
    
    # Update the min token length if necessary
    min_token_length = min(min_token_length, token_length)
    
    # Append the token length to the list
    token_lengths.append(token_length)

# Print the token lengths, max token length, and min token length
print("Token Lengths:", token_lengths)
print("Max Token Length:", max_token_length)
print("Min Token Length:", min_token_length)


NameError: name 'tokenizer' is not defined

In [2]:
%cd ../codebert-classification/code

/Users/adnan.alrawass/jupyter-1.0.0/project_snt/codebert-classification/code


/Users/adnan.alrawass/jupyter-1.0.0/myenv/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [8]:
%ls

__pycache__/              predictions.txt           train_glitch.jsonl
checkpoint-best-acc2/     pupet_advance/            train_glitch_with.jsonl
checkpoint-best-accc/     puppet-Akond-walliams/    untitled.txt
codebert-classification/  run-t.py                  untitled1.txt
enhance/                  run.py                    valid.jsonl
epoch3/                   test.jsonl                valid_glitch.jsonl
epoch6/                   test.log                  valid_glitch_with.jsonl
inference.sh              test_glitch.jsonl         with2/
lastansible/              test_glitch.jsonl.jsonl   withcontext/
logs/                     test_glitch_with.jsonl    withcontextClean/
model.py                  train.jsonl               withcontextPredicrion/
newdata/                  train.sh                  withoutcontextPredection/


In [15]:
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score

def calculate_metrics(predictions_file, labels_file):
    # Load predictions and labels from files
    with open(predictions_file, 'r') as f:
        predictions = [int(line.strip()) for line in f]
    with open(labels_file, 'r') as f:
        labels = [int(line.strip()) for line in f]

    # Calculate AUC
    auc = roc_auc_score(labels, predictions)

    # Convert predictions to binary labels
    predictions_binary = [1 if pred >= 0.5 else 0 for pred in predictions]

    # Calculate other metrics
    precision = precision_score(labels, predictions_binary)
    recall = recall_score(labels, predictions_binary)
    f1 = f1_score(labels, predictions_binary)
    accuracy = accuracy_score(labels, predictions_binary)

    return {
        'auc': auc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'accuracy': accuracy
    }

# File paths
predictions_file = "./pupet_advance/predictions.txt"
labels_file = "./pupet_advance/labels.txt"

# Calculate metrics
metrics = calculate_metrics(predictions_file, labels_file)

# Print metrics
print("Metrics:")
print(f"AUC: {metrics['auc']:.4f}")
print(f"Precision: {metrics['precision']:.4f}")
print(f"Recall: {metrics['recall']:.4f}")
print(f"F1-score: {metrics['f1']:.4f}")
print(f"Accuracy: {metrics['accuracy']:.4f}")

Metrics:
AUC: 0.6598
Precision: 0.7090
Recall: 0.6898
F1-score: 0.6992
Accuracy: 0.6638


In [9]:
#testing 
!python run.py\
    --output_dir=./puppet_william_akond_programing\
    --tokenizer_name=microsoft/codebert-base\
    --model_name_or_path=microsoft/codebert-base\
    --do_eval \
    --do_test \
    --train_data_file='/home/adnan/Desktop/training/codebert/train.jsonl'\
    --eval_data_file='/home/adnan/Desktop/training/codebert/valid.jsonl' \
    --test_data_file='/home/adnan/Desktop/training/codebert/test.jsonl' \
    --num_train_epochs 68 \
    --block_size 512 \
    --train_batch_size 8 \
    --eval_batch_size 8\
    --learning_rate 5e-5 \
    --max_grad_norm 1.0 \
    --seed 123456  2>&1 | tee test.log

07/09/2024 11:44:21 - WARNING - __main__ -   device: cuda, n_gpu: 1
Token indices sequence length is longer than the specified maximum sequence length for this model (2986 > 512). Running this sequence through the model will result in indexing errors
Vocabulary size: 50265
Training/evaluation parameters %s Namespace(adam_epsilon=1e-08, block_size=512, device=device(type='cuda'), do_eval=True, do_test=True, do_train=False, eval_batch_size=8, eval_data_file='/home/adnan/Desktop/training/codebert/valid.jsonl', learning_rate=5e-05, max_grad_norm=1.0, model_name_or_path='microsoft/codebert-base', n_gpu=1, num_train_epochs=68, output_dir='./puppet_william_akond_programing', seed=123456, test_data_file='/home/adnan/Desktop/training/codebert/test.jsonl', tokenizer_name='microsoft/codebert-base', train_batch_size=8, train_data_file='/home/adnan/Desktop/training/codebert/train.jsonl', warmup_steps=0, weight_decay=0.0)
*** Example ***
label: 0
input_tokens: ['<s>', 'notice', '(', 'mod', 'ular', '

In [11]:
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score

def calculate_metrics(predictions_file, labels_file):
    # Load predictions and labels from files
    with open(predictions_file, 'r') as f:
        predictions = [int(line.strip()) for line in f]
    with open(labels_file, 'r') as f:
        labels = [int(line.strip()) for line in f]

    # Calculate AUC
    auc = roc_auc_score(labels, predictions)

    # Convert predictions to binary labels
    predictions_binary = [1 if pred >= 0.5 else 0 for pred in predictions]

    # Calculate other metrics
    precision = precision_score(labels, predictions_binary)
    recall = recall_score(labels, predictions_binary)
    f1 = f1_score(labels, predictions_binary)
    accuracy = accuracy_score(labels, predictions_binary)

    return {
        'auc': auc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'accuracy': accuracy
    }

# File paths
predictions_file = "./pupet_advance/predictions.txt"
labels_file = "./pupet_advance/labels.txt"

# Calculate metrics
metrics = calculate_metrics(predictions_file, labels_file)

# Print metrics
print("Metrics:")
print(f"AUC: {metrics['auc']:.4f}")
print(f"Precision: {metrics['precision']:.4f}")
print(f"Recall: {metrics['recall']:.4f}")
print(f"F1-score: {metrics['f1']:.4f}")
print(f"Accuracy: {metrics['accuracy']:.4f}")

Metrics:
AUC: 0.6598
Precision: 0.7090
Recall: 0.6898
F1-score: 0.6992
Accuracy: 0.6638


In [13]:
def convert_examples_to_features(js, tokenizer, args):
    # source
    max_token_length = 0
    code = ' '.join(js['code'].split())
    window_size = args.block_size - 2  # Adjust window size as needed (consider max sequence length)

    # Handle cases where code length is less than window size
    if len(code.split()) < window_size:
        code_tokens = tokenizer.tokenize(code)[:args.block_size - 2]
        source_tokens = [tokenizer.cls_token] + code_tokens + [tokenizer.sep_token]
        source_ids = tokenizer.convert_tokens_to_ids(source_tokens)
        padding_length = args.block_size - len(source_ids)
        source_ids += [tokenizer.pad_token_id] * padding_length
        return InputFeatures(source_tokens, source_ids, js['label'])

    # Windowing for long code snippets
    features = []
    for start_idx in range(0, len(code), window_size):
        print("hello working")
        end_idx = min(start_idx + window_size, len(code))
        window_code = code[start_idx:end_idx]
        code_tokens = tokenizer.tokenize(window_code)[:args.block_size - 2]
        source_tokens = [tokenizer.cls_token] + code_tokens + [tokenizer.sep_token]
        source_ids = tokenizer.convert_tokens_to_ids(source_tokens)
        padding_length = args.block_size - len(source_ids)
        source_ids += [tokenizer.pad_token_id] * padding_length
        features.append(InputFeatures(source_tokens, source_ids, js['label']))

    return features  # Return a list of InputFeatures for each window

In [4]:
!python3 run.py \
    --output_dir=./puppet_new_akond_william \
    --tokenizer_name=microsoft/codebert-base \
    --model_name_or_path=microsoft/codebert-base \
    --do_train \
    --train_data_file='/Users/adnan.alrawass/jupyter-1.0.0/project_snt/puppet_william_akond_programing/train.jsonl'\
    --eval_data_file='/Users/adnan.alrawass/jupyter-1.0.0/project_snt/puppet_william_akond_programing/valid.jsonl' \
    --test_data_file='/Users/adnan.alrawass/jupyter-1.0.0/project_snt/puppet_william_akond_programing/test.jsonl' \
    --num_train_epochs 300 \
    --block_size 512\
    --train_batch_size 128 \
    --eval_batch_size  128\
    --learning_rate 2e-5 \
    --weight_decay 0.0001    \
    --max_grad_norm 1.0 \
    --seed 123456       \
    --warmup_steps=100

06/02/2024 12:11:09 - WARNING - __main__ -   device: cpu, n_gpu: 0
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Number of Embeddings (Vocabulary Size): 50265
Embedding Dimension (Size): 768 before change
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Training/evaluation parameters %s Namespace(adam_epsilon=1e-08, block_size=512, device=device(type='cpu'), do_eval=False, do_test=False, do_train=T

In [8]:
#after incresing batch size 
!python3 run.py \
    --output_dir=./puppet_new_akond_william \
    --tokenizer_name=microsoft/codebert-base \
    --model_name_or_path=microsoft/codebert-base \
    --do_train \
    --train_data_file='/Users/adnan.alrawass/jupyter-1.0.0/project_snt/puppet_william_akond_programing/train.jsonl'\
    --eval_data_file='/Users/adnan.alrawass/jupyter-1.0.0/project_snt/puppet_william_akond_programing/valid.jsonl' \
    --test_data_file='/Users/adnan.alrawass/jupyter-1.0.0/project_snt/puppet_william_akond_programing/test.jsonl' \
    --num_train_epochs 100 \
    --block_size 512\
    --train_batch_size 128 \
    --eval_batch_size  128\
    --learning_rate 2e-5 \
    --weight_decay 0.00001    \
    --max_grad_norm 1.0 \
    --seed 123456       \
    --warmup_steps=150

06/02/2024 15:36:33 - WARNING - __main__ -   device: cpu, n_gpu: 0
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Number of Embeddings (Vocabulary Size): 50265
Embedding Dimension (Size): 768 before change
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Training/evaluation parameters %s Namespace(adam_epsilon=1e-08, block_size=512, device=device(type='cpu'), do_eval=False, do_test=False, do_train=T

In [12]:
!ls

__pycache__                     test.jsonl
checkpoint-best-acc2            test.log
checkpoint-best-accc            test_glitch.jsonl
codebert-classification         test_glitch.jsonl.jsonl
enhance                         test_glitch_with.jsonl
epoch3                          train.jsonl
epoch6                          train.sh
inference.sh                    train_glitch.jsonl
lastansible                     train_glitch_with.jsonl
logs                            untitled.txt
model.py                        untitled1.txt
newdata                         valid.jsonl
predictions.txt                 valid_glitch.jsonl
pupet_advance                   valid_glitch_with.jsonl
puppet-Akond-walliams           with2
puppet_new_akond_william        withcontext
puppet_william_akond_programing withcontextClean
run-t.py                        withcontextPredicrion
run.py                          withoutcontextPredection


In [13]:
#testing 
!python run.py\
    --output_dir=./puppet_new_akond_william \
    --tokenizer_name=microsoft/codebert-base\
    --model_name_or_path=microsoft/codebert-base\
    --do_eval \
    --do_test \
    --train_data_file='/Users/adnan.alrawass/jupyter-1.0.0/project_snt/puppet_william_akond_programing/train.jsonl'\
    --eval_data_file='/Users/adnan.alrawass/jupyter-1.0.0/project_snt/puppet_william_akond_programing/valid.jsonl' \
    --test_data_file='/Users/adnan.alrawass/jupyter-1.0.0/project_snt/puppet_william_akond_programing/test.jsonl' \
    --num_train_epochs 100 \
    --block_size 512\
    --train_batch_size 128 \
    --eval_batch_size  128\
    --learning_rate 2e-5 \
    --weight_decay 0.00001    \
    --max_grad_norm 1.0 \
    --seed 123456       \
    --warmup_steps=150

/Users/adnan.alrawass/jupyter-1.0.0/myenv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
06/04/2024 08:32:55 - WARNING - __main__ -   device: cpu, n_gpu: 0
Number of Embeddings (Vocabulary Size): 50265
Embedding Dimension (Size): 768 before change
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Training/evaluation parameters %s Namespace(train_data_file='/Users/adnan.alrawass/jupyter-1.0.0/project_snt/puppet_william_akond_programing/train.jsonl', output_dir='./puppet_new_akon

In [16]:
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score

def calculate_metrics(predictions_file, labels_file):
    # Load predictions and labels from files
    with open(predictions_file, 'r') as f:
        predictions = [int(line.strip()) for line in f]
    with open(labels_file, 'r') as f:
        labels = [int(line.strip()) for line in f]

    # Calculate AUC
    auc = roc_auc_score(labels, predictions)

    # Convert predictions to binary labels
    predictions_binary = [1 if pred >= 0.5 else 0 for pred in predictions]

    # Calculate other metrics
    precision = precision_score(labels, predictions_binary)
    recall = recall_score(labels, predictions_binary)
    f1 = f1_score(labels, predictions_binary)
    accuracy = accuracy_score(labels, predictions_binary)

    return {
        'auc': auc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'accuracy': accuracy
    }

# File paths
predictions_file = "./puppet_new_akond_william/predictions.txt"
labels_file = "./puppet_new_akond_william/labels.txt"

# Calculate metrics
metrics = calculate_metrics(predictions_file, labels_file)

# Print metrics
print("Metrics:")
print(f"AUC: {metrics['auc']:.4f}")
print(f"Precision: {metrics['precision']:.4f}")
print(f"Recall: {metrics['recall']:.4f}")
print(f"F1-score: {metrics['f1']:.4f}")
print(f"Accuracy: {metrics['accuracy']:.4f}")

Metrics:
AUC: 0.6344
Precision: 0.6262
Recall: 0.6667
F1-score: 0.6458
Accuracy: 0.6344


In [17]:
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score

def calculate_metrics(predictions_file, labels_file):
    # Load predictions and labels from files
    with open(predictions_file, 'r') as f:
        predictions = [int(line.strip()) for line in f]
    with open(labels_file, 'r') as f:
        labels = [int(line.strip()) for line in f]

    # Calculate AUC
    auc = roc_auc_score(labels, predictions)

    # Convert predictions to binary labels
    predictions_binary = [1 if pred >= 0.5 else 0 for pred in predictions]

    # Calculate other metrics
    precision = precision_score(labels, predictions_binary)
    recall = recall_score(labels, predictions_binary)
    f1 = f1_score(labels, predictions_binary)
    accuracy = accuracy_score(labels, predictions_binary)

    return {
        'auc': auc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'accuracy': accuracy
    }

# File paths
predictions_file = "./puppet_new_akond_william/save_prede/predictions.txt"
labels_file = "./puppet_new_akond_william/save_prede/labels.txt"

# Calculate metrics
metrics = calculate_metrics(predictions_file, labels_file)

# Print metrics
print("Metrics:")
print(f"AUC: {metrics['auc']:.4f}")
print(f"Precision: {metrics['precision']:.4f}")
print(f"Recall: {metrics['recall']:.4f}")
print(f"F1-score: {metrics['f1']:.4f}")
print(f"Accuracy: {metrics['accuracy']:.4f}")

Metrics:
AUC: 0.6378
Precision: 0.6822
Recall: 0.7048
F1-score: 0.6933
Accuracy: 0.6468


In [28]:

#testing 
!python run.py\
    --output_dir=./puppet_new_akond_william \
    --tokenizer_name=microsoft/codebert-base\
    --model_name_or_path=microsoft/codebert-base\
    --do_eval \
    --do_test \
    --train_data_file='/Users/adnan.alrawass/jupyter-1.0.0/project_snt/puppet_william_akond_programing/train.jsonl'\
    --eval_data_file='/Users/adnan.alrawass/jupyter-1.0.0/project_snt/puppet_william_akond_programing/valid.jsonl' \
    --test_data_file='/Users/adnan.alrawass/jupyter-1.0.0/project_snt/puppet_advance//test.jsonl' \
    --num_train_epochs 100 \
    --block_size 512\
    --train_batch_size 128 \
    --eval_batch_size  128\
    --learning_rate 2e-5 \
    --weight_decay 0.00001    \
    --max_grad_norm 1.0 \
    --seed 123456       \
    --warmup_steps=150

/Users/adnan.alrawass/jupyter-1.0.0/myenv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
06/04/2024 10:36:21 - WARNING - __main__ -   device: cpu, n_gpu: 0
Number of Embeddings (Vocabulary Size): 50265
Embedding Dimension (Size): 768 before change
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Training/evaluation parameters %s Namespace(train_data_file='/Users/adnan.alrawass/jupyter-1.0.0/project_snt/puppet_william_akond_programing/train.jsonl', output_dir='./puppet_new_akon

In [35]:

#testing 
!python run.py\
    --output_dir=./puppet_new_akond_william \
    --tokenizer_name=microsoft/codebert-base\
    --model_name_or_path=microsoft/codebert-base\
    --do_eval \
    --do_test \
    --train_data_file='/Users/adnan.alrawass/jupyter-1.0.0/project_snt/puppet_william_akond_programing/train.jsonl'\
    --eval_data_file='/Users/adnan.alrawass/jupyter-1.0.0/project_snt/puppet_william_akond_programing/valid.jsonl' \
    --test_data_file='/Users/adnan.alrawass/jupyter-1.0.0/project_snt/puppet_advance//test.jsonl' \
    --num_train_epochs 100 \
    --block_size 512\
    --train_batch_size 128 \
    --eval_batch_size  128\
    --learning_rate 2e-5 \
    --weight_decay 0.00001    \
    --max_grad_norm 1.0 \
    --seed 123456       \
    --warmup_steps=150

/Users/adnan.alrawass/jupyter-1.0.0/myenv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
06/04/2024 13:33:06 - WARNING - __main__ -   device: cpu, n_gpu: 0
Number of Embeddings (Vocabulary Size): 50265
Embedding Dimension (Size): 768 before change
Token Lengths: [3, 5, 4, 5, 1, 4, 1, 3, 4, 3, 2, 3, 1, 4, 5, 3, 3, 2, 2, 5, 4, 4, 2, 3, 3, 3, 5, 1, 3, 1, 3, 5, 4, 5, 4, 1, 2, 3, 2, 4, 5, 3, 5, 1, 3, 4, 1, 4, 1, 6, 3, 5, 3, 4, 4, 5, 4, 5, 5, 6, 2, 6, 3, 4, 4, 4, 4, 5, 2, 4, 4, 6, 2, 1, 6, 2, 5, 5, 6, 4, 4, 5, 7, 2, 4, 3, 5, 5, 5, 6, 1, 3, 4, 4, 5, 5, 3, 6, 3, 5, 1, 5, 1, 5, 1, 2, 5, 6, 1, 3, 5, 2, 2, 1, 3, 6, 1, 3, 1, 1, 4, 2, 4, 4, 5, 4, 4, 3, 2, 5, 6, 1, 2, 3, 1, 8, 8, 7, 8, 1, 6, 4, 8, 4, 5, 6, 5, 6, 7, 8, 6, 3, 5, 8, 3, 2, 5, 5, 3, 5, 4, 5, 3, 2, 6, 5, 3, 6, 11, 4, 2, 5, 5, 5, 5, 3, 1, 5, 4, 2,

In [ ]:
def convert_examples_to_features(js, tokenizer, args):
    # source
    max_token_length = 0
    code = ' '.join(js['code'].split())
    window_size = args.block_size - 2  # Adjust window size as needed (consider max sequence length)

    # Handle cases where code length is less than window size
    if len(code.split()) < window_size:
        print(code,"test i shater")
        code_tokens = tokenizer.tokenize(code)[:args.block_size - 2]
        source_tokens = [tokenizer.cls_token] + code_tokens + [tokenizer.sep_token]
        source_ids = tokenizer.convert_tokens_to_ids(source_tokens)
        padding_length = args.block_size - len(source_ids)
        source_ids += [tokenizer.pad_token_id] * padding_length
        return InputFeatures(source_tokens, source_ids, js['label'])

    # Windowing for long code snippets
    features = []
    for start_idx in range(0, len(code), window_size):

        end_idx = min(start_idx + window_size-2, len(code))
        window_code = code[start_idx:end_idx]
        code_tokens = tokenizer.tokenize(window_code)[:args.block_size - 2]
        source_tokens = [tokenizer.cls_token] + code_tokens + [tokenizer.sep_token]
        source_ids = tokenizer.convert_tokens_to_ids(source_tokens)
        padding_length = args.block_size - len(source_ids)
        source_ids += [tokenizer.pad_token_id] * padding_length
        features.append(InputFeatures(source_tokens, source_ids, js['label']))

    return features  # Return a list of InputFeatures for each window

In [3]:
#after incresing batch size 
!python3 run.py \
    --output_dir=./puppet_new_akond_william \
    --tokenizer_name=microsoft/codebert-base \
    --model_name_or_path=microsoft/codebert-base \
    --do_train \
    --train_data_file='/Users/adnan.alrawass/jupyter-1.0.0/project_snt/puppet_william_akond_programing/train.jsonl'\
    --eval_data_file='/Users/adnan.alrawass/jupyter-1.0.0/project_snt/puppet_william_akond_programing/valid.jsonl' \
    --test_data_file='/Users/adnan.alrawass/jupyter-1.0.0/project_snt/puppet_william_akond_programing/test.jsonl' \
    --num_train_epochs 50\
    --block_size 512\
    --train_batch_size 64 \
    --eval_batch_size  64\
    --learning_rate 1e-5 \
    --max_grad_norm 1.0 \
    --seed 123456       \
    --warmup_steps=150

06/05/2024 00:27:57 - WARNING - __main__ -   device: mps, n_gpu: 1
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Vocabulary size: 50265
Training/evaluation parameters %s Namespace(adam_epsilon=1e-08, block_size=512, device=device(type='mps'), do_eval=False, do_test=False, do_train=True, eval_batch_size=64, eval_data_file='/Users/adnan.alrawass/jupyter-1.0.0/project_snt/puppet_william_akond_programing/valid.jsonl', learning_rate=1e-05, max_grad_norm=1.0, model_name_or_path='microsoft/codebert-base', n_gpu=1, num_train_epochs=50, output_dir='./puppet_new_akond_william', seed=123456, test_data_file='/Users/adnan.alrawass/jupyter-1.0.0/project_snt/puppet_william_akond_programing/test.jsonl', tokenizer_name='microsoft/c

In [4]:
#testing 
!python run.py\
    --output_dir=./puppet_new_akond_william \
    --tokenizer_name=microsoft/codebert-base\
    --model_name_or_path=microsoft/codebert-base\
    --do_eval \
    --do_test \
    --train_data_file='/Users/adnan.alrawass/jupyter-1.0.0/project_snt/puppet_william_akond_programing/train.jsonl'\
    --eval_data_file='/Users/adnan.alrawass/jupyter-1.0.0/project_snt/puppet_william_akond_programing/valid.jsonl' \
    --test_data_file='/Users/adnan.alrawass/jupyter-1.0.0/project_snt/puppet_william_akond_programing/test.jsonl' \
    --num_train_epochs 50 \
    --block_size 512\
    --train_batch_size 64 \
    --eval_batch_size  64\
    --learning_rate 1e-5 \
    --max_grad_norm 1.0 \
    --seed 123456       \
    --warmup_steps=150

/Users/adnan.alrawass/jupyter-1.0.0/myenv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
06/05/2024 05:16:13 - WARNING - __main__ -   device: mps, n_gpu: 1
Vocabulary size: 50265
Training/evaluation parameters %s Namespace(train_data_file='/Users/adnan.alrawass/jupyter-1.0.0/project_snt/puppet_william_akond_programing/train.jsonl', output_dir='./puppet_new_akond_william', eval_data_file='/Users/adnan.alrawass/jupyter-1.0.0/project_snt/puppet_william_akond_programing/valid.jsonl', test_data_file='/Users/adnan.alrawass/jupyter-1.0.0/project_snt/puppet_william_akond_programing/test.jsonl', model_name_or_path='microsoft/codebert-base', tokenizer_name='microsoft/codebert-base', block_size=512, do_train=False, do_eval=True, do_test=True, train_batch_size=64, eval_batch_size=64, learning_rate=1e-05, 

In [5]:
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score

def calculate_metrics(predictions_file, labels_file):
    # Load predictions and labels from files
    with open(predictions_file, 'r') as f:
        predictions = [int(line.strip()) for line in f]
    with open(labels_file, 'r') as f:
        labels = [int(line.strip()) for line in f]

    # Calculate AUC
    auc = roc_auc_score(labels, predictions)

    # Convert predictions to binary labels
    predictions_binary = [1 if pred== 1 else 0 for pred in predictions]

    # Calculate other metrics
    precision = precision_score(labels, predictions_binary)
    recall = recall_score(labels, predictions_binary)
    f1 = f1_score(labels, predictions_binary)
    accuracy = accuracy_score(labels, predictions_binary)

    return {
        'auc': auc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'accuracy': accuracy
    }

# File paths
predictions_file = "./puppet_new_akond_william/predictions.txt"
labels_file = "./puppet_new_akond_william/labels.txt"

# Calculate metrics
metrics = calculate_metrics(predictions_file, labels_file)

# Print metrics
print("Metrics:")N
print(f"AUC: {metrics['auc']:.4f}")
print(f"Precision: {metrics['precision']:.4f}")
print(f"Recall: {metrics['recall']:.4f}")
print(f"F1-score: {metrics['f1']:.4f}")
print(f"Accuracy: {metrics['accuracy']:.4f}")

Metrics:
AUC: 0.6361
Precision: 0.6163
Recall: 0.7211
F1-score: 0.6646
Accuracy: 0.6361
